In [1]:
import torch
import numpy
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class Rectangle:
    
    def __init__(self):
        self.length = 6
        self.width = 4

In [4]:
class Square(Rectangle):

    def __init__(self):
        super().__init__()

    def area(self):
        return self.length * self.width
    
square = Square()
print(square.area())

24


In [5]:
class Rectangle:

    def __init__(self, con):
        self.length = 6 + con
        self.width = 4

In [6]:
class Square(Rectangle):

    def __init__(self, con):
        super().__init__(con)

    def area(self):
        return self.length*self.width
    
    
    
square = Square(7)
print(square.area())

52


In [7]:
class Cube(Rectangle):

    def __init__(self, height, con): 
        super().__init__(con)
        self.height = height

    def volume(self):
        return self.length*self.width*self.height


cube = Cube(height=3, con=4)
print(cube.volume())

120


In [8]:
# PyTorch Model Example

class Classification(nn.Module):
    
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        
        self.layer_1 = nn.Linear(in_features=input_shape, out_features=hidden_units)
        self.layer_2 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.activation = nn.ReLU()
        self.classifier = nn.Linear(in_features=hidden_units, out_features=output_shape)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.activation(x)
        x = self.classifier(x)
        return x

In [9]:
data = torch.tensor([2.,1.,10.])
model = Classification(3, 4, 1)

In [10]:
model(data)

tensor([-0.5622], grad_fn=<ViewBackward0>)